# 프로젝트 개요
- 강의명 : 2022년 K-디지털 직업훈련(Training) 사업 - AI데이터플랫폼을 활용한 빅데이터 분석전문가 과정
- 교과목명 : 빅데이터 분석 및 시각화, AI개발 기초, 인공지능 프로그래밍
- 프로젝트 주제 : Spaceship Titanic 데이터를 활용한 탑승유무 분류모형 개발 
- 프로젝트 마감일 : 2022년 4월 12일 화요일
- 수강생명 : 오승은

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 라이브러리 불러오기

- 프로젝트에 쓰일 라이브러리를 모두 불러온다

In [ ]:
# Core
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
from imblearn.over_sampling import SMOTE
import itertools
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
import time



# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, plot_confusion_matrix, plot_roc_curve, roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.utils import resample

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

# 데이터 준비

- 훈련 데이터는 8693개, 테스트 데이터는 4277개이다
- 독립변수는 13개, 종속변수(타겟값)은 1개임을 확인했다

In [ ]:
# Save to df
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')

# Shape and preview
print('Train set shape:', train.shape)
print('Test set shape:', test.shape)
train.head()

## 결측치 확인

In [ ]:
print('TRAIN SET MISSING VALUES:')
print(train.isna().sum())
print('')
print('TEST SET MISSING VALUES:')
print(test.isna().sum())

- 거의 모든 데이터에 결측지 존재

## 중복 확인

- 중복된 row 값을 찾아주는 duplicated 메서드로 중복치를 확인해보자

In [ ]:
print(f'Duplicates in train set: {train.duplicated().sum()}, ({np.round(100*train.duplicated().sum()/len(train),1)}%)')
print('')
print(f'Duplicates in test set: {test.duplicated().sum()}, ({np.round(100*test.duplicated().sum()/len(test),1)}%)')

- Cardinality of features (특성들의 중복 정도를 알아보자)
- 중복도가 ‘낮으면’ 카디널리티가 ‘높다’고 표현/중복도가 ‘높으면’ 카디널리티가 ‘낮다’고 표현

In [ ]:
train.nunique()

- 훈련 데이터들의 유형을 확인해본다

In [ ]:
train.dtypes

- train 데이터는 bool(1), float64(6), object(7)개로 구성되어 있다
- 머신러닝 모델 학습을 시키기 위해서 데이터들을 숫자형 타입으로 바꿀 필요가 있음 

# EDA

- 타겟(종속변수)인 transported 분포 확인

In [ ]:
# Figure size
plt.figure(figsize=(6,6))

# Pie plot 
train['Transported'].value_counts().plot.pie(explode=[0.1,0.1], autopct='%1.1f%%', shadow=True, textprops={'fontsize':16}).set_title("Target distribution", backgroundcolor='lightgray',pad=20)


- 타겟 분포가 매우 고르므로 under sampling/ over sampling 하지 않아도 된다

- 연속형 변수들이 transported와 연관있는지 그래프를 그려 확인해보기

In [ ]:
# Figure size
plt.figure(figsize=(10,4))

# Histogram
sns.histplot(data=train, x='Age', hue='Transported', binwidth=1, kde=True)

# Aesthetics
plt.title('Age distribution')
plt.xlabel('Age (years)')

- 0-18세는 tansported되는 경향이 높고, 18-25세는 transported 되지 않는 경향이 높고, 25세 이후로는 거의 비슷한 경향을 보인다
- child(0-18), adolescent(18-25), adult(25-)라는 새로운 특성을 만들자

In [ ]:
# Expenditure features
exp_feats=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Plot expenditure features
fig=plt.figure(figsize=(10,20))
for i, var_name in enumerate(exp_feats):
    # Left plot
    ax=fig.add_subplot(5,2,2*i+1)
    sns.histplot(data=train, x=var_name, axes=ax, bins=30, kde=False, hue='Transported')
    ax.set_title(var_name)
    
    # Right plot (truncated)
    ax=fig.add_subplot(5,2,2*i+2)
    sns.histplot(data=train, x=var_name, axes=ax, bins=30, kde=True, hue='Transported')
    plt.ylim([0,100])
    ax.set_title(var_name)
fig.tight_layout()
plt.show()

- transported된 사람들은 돈을 덜 쓴 경향이 있다
- outlier가 존재
- 소비 분포는 기하급수적으로 낮아진다
- 대부분의 사람들은 돈을 거의 쓰지 않았다
- insights : 
    + 5가지 시설에 대한 총 지출을 추적하는 새로운 변수 만들기
    + 지출을 했는지/ 안 했는지 binary 변수 만들기

- 범주형 변수들이 transported와 연관있는지 그래프를 그려 확인해보기

In [ ]:
# Categorical features
cat_feats=['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

# Plot categorical features
fig=plt.figure(figsize=(10,16))
for i, var_name in enumerate(cat_feats):
    ax=fig.add_subplot(4,1,i+1)
    sns.countplot(data=train, x=var_name, axes=ax, hue='Transported')
    ax.set_title(var_name)
fig.tight_layout() 
plt.show() 

- vip 특성은 transported의 차이가 없어서 유용한 특성이 아니라고 보여진다 -> overfitting을 방지하기 위해 이 특성을 삭제해야한다
- cryosleep은 transported의 차이가 나기 때문에 유용한 특성으로 보인다 

- 질적 변수들은 아직 시각화하지 못하므로 qual_feats으로 묶고 데이터 확인하기

In [ ]:
# Qualitative features
qual_feats=['PassengerId', 'Cabin' ,'Name']

# Preview qualitative features
train[qual_feats].head()

- PassengerId는 그룹_그룹 인원수 형태로 구성되어 있다
- Cabin은 갑판/번호/사이드 형태로 구성되어 있다
--> 그러므로
- PassengerId로부터 그룹/그룹 사이즈를 추출할 수 있다
- Cabin으로부터 갑판, 번호, 사이드를 추출할 수 있다
- Name으로부터 성을 추출할 수 있다

# 결측치 처리

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(train.isna().T, cmap='summer')
plt.title('Heatmap of missing values')

- 결측치가 전체 데이터의 2%를 차지하고 있으며, 대부분 동시에 발생하지는 않는 것 같다
(결측치의 비중이 낮다)

In [ ]:
train['na_count']=train.isna().sum(axis=1)
plt.figure(figsize=(10,4))
sns.countplot(data=train, x='na_count', hue='Transported')
plt.title('Number of missing entries by passenger')
train.drop('na_count', axis=1, inplace=True)

- 결측치들이 대부분 결과랑 분리되어 있으므로 이를 제거하기 보단 다른 적절한 값으로 채워주는 것이 합리적으로 보인다

- 연속형 데이터인 age의 결측치는 중앙값으로 채운다

In [ ]:
# Impute median (for continuous data)
train['Age'].fillna(train['Age'].median(), inplace=True)
test['Age'].fillna(train['Age'].median(), inplace=True) # be careful of data leakage

- mode()를 사용해 가장 많이 등장하는 값을 알아본다

In [ ]:
# Find mode of each categorical feature
train[['HomePlanet','CryoSleep','Destination','VIP']].mode()

- 각 데이터들의 결측치를 최빈값으로 채워준다

In [ ]:
# Impute most frequent category (for categorical data)
train['HomePlanet'].fillna('Earth', inplace=True)
test['HomePlanet'].fillna('Earth', inplace=True)

train['CryoSleep'].fillna(False, inplace=True)
test['CryoSleep'].fillna(False, inplace=True)

train['Destination'].fillna('TRAPPIST-1e', inplace=True)
test['Destination'].fillna('TRAPPIST-1e', inplace=True)

train['VIP'].fillna(False, inplace=True)
test['VIP'].fillna(False, inplace=True)

# Impute 0's (mode) because we will create a categorical column from this later
for col in exp_feats:
    train.loc[train[col].isna(),col]=0
    test.loc[test[col].isna(),col]=0

- 질적 데이터인 Cabin과 Name 결측치 처리하기 (인위적으로 없는 값으로 채워준다)

In [ ]:
# Impute outliers (for qualitative data)
train['Cabin'].fillna('Z/9999/Z', inplace=True)
test['Cabin'].fillna('Z/9999/Z', inplace=True)

train['Name'].fillna('No Name', inplace=True)
test['Name'].fillna('No Name', inplace=True)

# 특성 공학

- EDA를 단계를 참고하여 child(0-18), adolescent(18-25), adult(25-)라는 새로운 특성을 만들기

In [ ]:
# New features - training set
train['Under_18']=(train['Age']<18).astype(int)
train['18_to_25']=((train['Age']>=18) & (train['Age']<=25)).astype(int)
train['Over_25']=(train['Age']>25).astype(int)

# New features - test set
test['Under_18']=(test['Age']<18).astype(int)
test['18_to_25']=((test['Age']>=18) & (test['Age']<=25)).astype(int)
test['Over_25']=(test['Age']>25).astype(int)

# Plot distribution of new features
train['Age_plot']=train['Under_18']+2*train['18_to_25']+3*train['Over_25']
plt.figure(figsize=(10,4))
g=sns.countplot(data=train, x='Age_plot', hue='Transported')
plt.title('Age status distribution')
g.set_xticklabels(['Under 18', '18-25', 'Over 25'])
train.drop('Age_plot', axis=1, inplace=True)

- 총지출을 계산하고 지출이 없는 승객들을 확인해보기

In [ ]:
# New features - training set
train['Expenditure_luxury']=train['RoomService']+train['Spa']+train['VRDeck']
train['Expenditure_necessary']=train['FoodCourt']+train['ShoppingMall']
train['No_luxury_spending']=(train['Expenditure_luxury']==0).astype(int)
train['No_necessary_spending']=(train['Expenditure_necessary']==0).astype(int)

# New features - test set
test['Expenditure_luxury']=test['RoomService']+test['Spa']+test['VRDeck']
test['Expenditure_necessary']=test['FoodCourt']+test['ShoppingMall']
test['No_luxury_spending']=(test['Expenditure_luxury']==0).astype(int)
test['No_necessary_spending']=(test['Expenditure_necessary']==0).astype(int)

# Plot distribution of new features
fig=plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
sns.histplot(data=train, x='Expenditure_luxury', hue='Transported', binwidth=100)
plt.title('Total luxury expenditure')
plt.xlim([0,6000])
plt.ylim([0,300])

plt.subplot(2,2,2)
sns.countplot(data=train, x='No_luxury_spending', hue='Transported')
plt.title('No luxury spending')

plt.subplot(2,2,3)
sns.histplot(data=train, x='Expenditure_necessary', hue='Transported', binwidth=100)
plt.title('Total necessary expenditure')
plt.xlim([0,6000])
plt.ylim([0,300])

plt.subplot(2,2,4)
sns.countplot(data=train, x='No_necessary_spending', hue='Transported')
plt.title('No necessary spending')
fig.tight_layout()

- PassengerId로부터 group, group_size 추출하기

In [ ]:
# New features - training set
train['Group'] = train['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)
train['Group_size']=train['Group'].map(lambda x: train['Group'].value_counts()[x])

# New features - test set
test['Group'] = test['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)
test['Group_size']=test['Group'].map(lambda x: test['Group'].value_counts()[x])

# Plot distribution of new features
plt.figure(figsize=(20,4))
plt.subplot(1,2,1)
sns.histplot(data=train, x='Group', hue='Transported', binwidth=1)
plt.title('Group')

plt.subplot(1,2,2)
sns.countplot(data=train, x='Group_size', hue='Transported')
plt.title('Group size')
fig.tight_layout()

- group_size=1일 경우, solo라는 feature추가

In [ ]:
train['Solo'] = (train['Group_size']==1).astype(int)
test['Solo'] = (test['Group_size']==1).astype(int)

plt.figure(figsize=(10,6))
sns.countplot(data=train,x='Solo',hue='Transported')
plt.title('Passenger alone or not')
plt.ylim([0,3000])
plt.show()

- 혼자(솔로)인 승객들이 transported되는 경향이 적다

- Cabin으로 부터 deck,number,side 추출하기

In [ ]:
# New features - training set
train['Cabin_deck'] = train['Cabin'].apply(lambda x: x.split('/')[0])
train['Cabin_number'] = train['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
train['Cabin_side'] = train['Cabin'].apply(lambda x: x.split('/')[2])

# New features - test set
test['Cabin_deck'] = test['Cabin'].apply(lambda x: x.split('/')[0])
test['Cabin_number'] = test['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
test['Cabin_side'] = test['Cabin'].apply(lambda x: x.split('/')[2])

# Plot distribution of new features
fig=plt.figure(figsize=(10,12))
plt.subplot(3,1,1)
sns.countplot(data=train, x='Cabin_deck', hue='Transported', order=['A','B','C','D','E','F','G','T','Z'])
plt.title('Cabin deck')

plt.subplot(3,1,2)
sns.histplot(data=train, x='Cabin_number', hue='Transported',binwidth=20)
plt.vlines(300, ymin=0, ymax=200, color='black')
plt.vlines(600, ymin=0, ymax=200, color='black')
plt.vlines(900, ymin=0, ymax=200, color='black')
plt.vlines(1200, ymin=0, ymax=200, color='black')
plt.vlines(1500, ymin=0, ymax=200, color='black')
plt.vlines(1800, ymin=0, ymax=200, color='black')
plt.title('Cabin number')
plt.xlim([0,2000])

plt.subplot(3,1,3)
sns.countplot(data=train, x='Cabin_side', hue='Transported')
plt.title('Cabin side')
fig.tight_layout()

- Cabin_number은 300개씩 그룹화 되어있는 것 같다 -> 범주형으로 만들 수 있을듯
- The cabin deck 'T'는 5개 샘플밖에 존재하지않아서 아웃라이어 같다 


In [ ]:
# New features - training set
train['Cabin_region1']=(train['Cabin_number']<300).astype(int)   # one-hot encoding
train['Cabin_region2']=((train['Cabin_number']>=300) & (train['Cabin_number']<600)).astype(int)
train['Cabin_region3']=((train['Cabin_number']>=600) & (train['Cabin_number']<900)).astype(int)
train['Cabin_region4']=((train['Cabin_number']>=900) & (train['Cabin_number']<1200)).astype(int)
train['Cabin_region5']=((train['Cabin_number']>=1200) & (train['Cabin_number']<1500)).astype(int)
train['Cabin_region6']=((train['Cabin_number']>=1500) & (train['Cabin_number']<1800)).astype(int)
train['Cabin_region7']=((train['Cabin_number']>=1800) & (train['Cabin_number']<2100)).astype(int)

# New features - test set
test['Cabin_region1']=(test['Cabin_number']<300).astype(int)   # one-hot encoding
test['Cabin_region2']=((test['Cabin_number']>=300) & (test['Cabin_number']<600)).astype(int)
test['Cabin_region3']=((test['Cabin_number']>=600) & (test['Cabin_number']<900)).astype(int)
test['Cabin_region4']=((test['Cabin_number']>=900) & (test['Cabin_number']<1200)).astype(int)
test['Cabin_region5']=((test['Cabin_number']>=1200) & (test['Cabin_number']<1500)).astype(int)
test['Cabin_region6']=((test['Cabin_number']>=1500) & (test['Cabin_number']<1800)).astype(int)
test['Cabin_region7']=((test['Cabin_number']>=1800) & (test['Cabin_number']<2100)).astype(int)

# Plot distribution of new features
plt.figure(figsize=(10,4))
train['Cabin_regions_plot']=(train['Cabin_region1']+2*train['Cabin_region2']+3*train['Cabin_region3']+4*train['Cabin_region4']+5*train['Cabin_region5']+6*train['Cabin_region6']+7*train['Cabin_region7']).astype(int)
sns.countplot(data=train, x='Cabin_regions_plot', hue='Transported')
plt.title('Cabin regions')
train.drop('Cabin_regions_plot', axis=1, inplace=True)

- Name에서 가족이름 추출

In [ ]:
# New features - training set
# str.split().str[-1] : 문자열을 나눈 후 마지막열(-1)만 가져와라
train['Surname']=train['Name'].str.split().str[-1]
train['Family_size']=train['Surname'].map(lambda x: train['Surname'].value_counts()[x])

# New features - test set
test['Surname']=test['Name'].str.split().str[-1]
test['Family_size']=test['Surname'].map(lambda x: test['Surname'].value_counts()[x])

# Set outliers (no name) to have no family
train.loc[train['Family_size']==200,'Family_size']=0
test.loc[test['Family_size']==200,'Family_size']=0

# New feature distribution
plt.figure(figsize=(10,4))
sns.countplot(data=train, x='Family_size', hue='Transported')
plt.title('Family size')

- Name에서 성별 추출

# 전처리

- 필요없는 특성들('PassengerId', 'Cabin', 'Name', 'Surname', 'Age', 'Group', 'Cabin_number') 을 drop 시켜준다

In [ ]:
# Drop qualitative/redundant/high cardinality features
train.drop(['PassengerId', 'Cabin', 'Name', 'Surname', 'Age', 'Group', 'Cabin_number'], axis=1, inplace=True)
test.drop(['PassengerId', 'Cabin', 'Name', 'Surname', 'Age', 'Group', 'Cabin_number'], axis=1, inplace=True)

# Preview resulting training set
train.head()

- 원 데이터가 섞이는걸 방지하기 위해 복제해준다

In [ ]:
y=train['Transported'].copy().astype(int)
X=train.drop('Transported', axis=1).copy()
X_test=test.copy()

- encoding & scaling
- Pipeline()
    + 서로 다른 매개변수를 설정하면서 교차 검증을 할 수 있는 단계들을 조합하기 위해 사용
    + data leakage 방지

In [ ]:
# Indentify numerical and categorical columns
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]

# Scale numerical data to have mean=0 and variance=1
numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# One-hot encode categorical data
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(drop='if_binary', handle_unknown='ignore',sparse=False))])

# Combine preprocessing
ct = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)],
        remainder='passthrough')

# Apply preprocessing
X = ct.fit_transform(X)
X_test = ct.transform(X_test)

# Print new shape
print('Training set shape:', X.shape)

## PCA

- 주성분 분석을 시행 (주성분 갯수는 3개로 지정)
- (8693, 3) 크기의 배열로 차원 축소
- PCA의 explained_variance_ratio_ 메서드를 사용하여 분산 비율(고유값/총 고유값)을 구한다
    + 설명된 분산은 특정 주성분을 사용하여 설명된 정보를 나타낸다
- 3차원 산점도를 사용하여 주성분들의 분포를 시각화한다

In [ ]:
pca = PCA(n_components=3)
components = pca.fit_transform(X)
print(components.shape)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=y, size=0.1*np.ones(len(X)), opacity = 1,
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
    width=800, height=500
)
fig.show()

- 주성분이 원본데이터의 32.77%만큼만 설명하고 있다

- 적절한 주성분의 갯수 알아보기

In [ ]:
# Explained variance (how important each additional principal component is)
pca = PCA().fit(X)
fig, ax = plt.subplots(figsize=(10,4))
xi = np.arange(1, 1+X.shape[1], step=1)
yi = np.cumsum(pca.explained_variance_ratio_)
plt.plot(xi, yi, marker='o', linestyle='--', color='b')

# Aesthetics
plt.ylim(0.0,1.1)
plt.xlabel('Number of Components')
plt.xticks(np.arange(1, 1+X.shape[1], step=2))
plt.ylabel('Cumulative variance (%)')
plt.title('Explained variance by each component')
plt.axhline(y=1, color='r', linestyle='-')
plt.text(0.5, 0.85, '100% cut-off threshold', color = 'red')
ax.grid(axis='x')

- 15개 이상의 주성분이 분산의 대부분을 표현하고 있다

- 검정을 위해 데이터 세트 분리한다

In [ ]:
# Train-validation split
X_train, X_valid, y_train, y_valid = train_test_split(X,y,stratify=y,train_size=0.8,test_size=0.2,random_state=0)

# 모델 선택

In [ ]:
# Classifiers
classifiers = {
    "LogisticRegression" : LogisticRegression(random_state=0),
    "KNN" : KNeighborsClassifier(),
    "RandomForest" : RandomForestClassifier(random_state=0),
    #"XGBoost" : XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss'), # XGBoost takes too long
    "LGBM" : LGBMClassifier(random_state=0),
    "CatBoost" : CatBoostClassifier(random_state=0, verbose=False),
}

# Grids for grid search
LR_grid = {'penalty': ['l1','l2'],
           'C': [0.25, 0.5, 0.75, 1, 1.25, 1.5],
           'max_iter': [50, 100, 150]}

KNN_grid = {'n_neighbors': [3, 5, 7, 9],
            'p': [1, 2]}



RF_grid = {'n_estimators': [50, 100, 150, 200, 250, 300],
        'max_depth': [4, 6, 8, 10, 12]}

boosted_grid = {'n_estimators': [50, 100, 150, 200],
        'max_depth': [4, 8, 12],
        'learning_rate': [0.05, 0.1, 0.15]}




# Dictionary of all grids
grid = {
    "LogisticRegression" : LR_grid,
    "KNN" : KNN_grid,
    "RandomForest" : RF_grid,
    "XGBoost" : boosted_grid,
    "LGBM" : boosted_grid,
    "CatBoost" : boosted_grid,
}

- 그리드 서치로 모델 훈련

In [ ]:
i = 0
clf_best_params=classifiers.copy()
valid_scores=pd.DataFrame({'Classifier': classifiers.keys(), 'Validation accuracy': np.zeros(len(classifiers)), 'Training time': np.zeros(len(classifiers))})
for key, classifier in classifiers.items():
    start = time.time()
    clf = GridSearchCV(estimator=classifier, param_grid=grid[key], n_jobs=-1, cv=None)
    
    # train and score
    clf.fit(X_train,y_train)
    valid_scores.iloc[i,1]=clf.score(X_valid, y_valid)
    
    # save trained model
    clf_best_params[key]=clf.best_params_
    
    # print iteration and training time
    stop = time.time()
    valid_scores.iloc[i:2] = np.round((start-stop)/60,2)
    
    print('Model:', key)
    print('Training time (mins):', valid_scores.iloc[i,2])
    print('')
    i+=1
    

In [ ]:
# Show results
valid_scores

- 이 결과를 보니 모델링 마지막 단계에서 RandomForest, LGBM , CatBoost를 시행하는 것이 좋아보인다

In [ ]:
# Show best parameters from grid search
clf_best_params

# 모델링

- 전체 훈련세트에 대해 가장 좋은 모델을 훈련시킨다
- 마지막으로 교차 검증 전체 교육 세트에 대해 최고의 모델을 교육및 앙상블 예측을 함께 사용한다

In [ ]:
# Classifiers
best_classifiers = {
    "RandomForest" : RandomForestClassifier(**clf_best_params["RandomForest"], random_state=0),
    "LGBM" : LGBMClassifier(**clf_best_params["LGBM"], random_state=0),
    "CatBoost" : CatBoostClassifier(**clf_best_params["CatBoost"], verbose=False, random_state=0),
}

- 교차검증과 앙상블 예측 시행하기

In [ ]:
# Number of folds in cross validation
FOLDS=10

preds=np.zeros(len(X_test))
for key, classifier in best_classifiers.items():
    start = time.time()
    
    # 5-fold cross validation
    cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=0)
    
    score=0
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        # Get training and validation sets
        X_train, X_valid = X[train_idx], X[val_idx]
        y_train, y_valid = y[train_idx], y[val_idx]

        # Train model
        clf = classifier
        clf.fit(X_train, y_train)

        # Make predictions and measure accuracy
        preds += clf.predict_proba(X_test)[:,1]
        score += clf.score(X_valid, y_valid)

    # Average accuracy    
    score=score/FOLDS
    
    # Stop timer
    stop = time.time()

    # Print accuracy and time
    print('Model:', key)
    print('Average validation accuracy:', np.round(100*score,2))
    print('Training time (mins):', np.round((stop - start)/60,2))
    print('')
    
# Ensemble predictions
preds=preds/(FOLDS*len(best_classifiers))

- 혼동행렬 (Confusion Matrix) 
    + 주로 분류문제에서 모델의 성능을 평가할 때 혼동행렬 형태를 사용한다
    + 4가지 개념 -> 앞이 T면 예측이 맞았음/ F이면 예측이 틀렸음을 의미 , 뒤가 P면 예측을 양성으로 했음/ N이면 예측을 음성으로 했음을 의미
        + TP/TN/FP/FN
        + TP는 true positive로 양성(1)으로 예측했는데 양성(1)일 경우를 말한다
        + TF는 true negative로 음성(0)으로 예측했는데 음성(0)일 경우를 말한다
        + FP는 false positive로 양성(1)으로 예측했는데 음성(0)일 경우를 말한다
        + FN는 false negative로 음성(0)으로 예측했는데 양성(1)일 경우를 말한다  
    + 평가지표 5가지
        + 정확도 = 전체에서 TP와 TF가 차지하는 비율 = (TP+TF) / (TP+TF+FP+FN)
        + 정밀도 = 양성으로 예측한 경우 중에서 실제 양성인 비율 = TP / (TP+FP)
        + 재현율 = 실제 양성 중에 제대로 예측한 비율 = TP / (TP+FN)
        + 특이도 = 실제 음성 중에 제대로 예측한 비율 = TN / (TN+FP)
        + F1 score = 정밀도와 재현율의 조화평균 = (2*정밀도*재현율) / (정밀도+재현율)
- 분류기준 값 (cut off)
    + 집단 1에 속할 확률을 분류기준 값과 비교한다
    + 확률이 분류기준보다 크면 양성으로 예측, 작으면 음성으로 예측한다
    + 두 집단을 분류하는 모형의 분류기준 값은 일반적으로 0.5 이다
    + 다른 분류기준 값을 사용하는 경우도 있다
        + 오분류율은 증가할 수 있지만 양성을 정확하게 분류하는 것이 중요하기 때문이다
        + 즉, 오분류비용이 비대칭일 경우 0.5가 아닌 다른 분류기준 값을 사용해야 한다
        
- 리프트 도표
    + 관측치가 목표집단에 속하는지 판별하는 분류모형의 성과 예측에 쓰인다
    + 단지 두 집단으로 분류하지 않을 때 리프트 곡선은 유용하다
    + 리프트 곡선은 상대적으로 적은 사례로 높은 응답자 비율을 찾아 가장 좋은 성과를 내는 부분을 찾고자 할 때 쓰인다
    + 예) 우편광고물에 응답할 가능성이 높은 사람을 판별해주는 데이터 마이닝 모형 적용
    
- ROC 곡선
    + 이진분류에 대한 성능 평가 기법
    + 이 곡선의 y축은 민감도 혹은 진양성률(true positive rate)을 나타내고, x축의 변수는 분류기준 값에 따라 계산된 특이도 혹은 진음성률(true negative rate)을 이용하여 계산된 값(1-특이도)으로 나타낸다
    + x축은 음성(0)으로 예측했는데 양성(1)일 비율이고, y축은 양성(1)으로 예측했는데 양성(1)일 비율이 된다. 진짜로 예측한 값들 중에서 실제로도 진짜일 경우가 실제로 가짜일 경우보다 높아야 AUC가 높게 나온다.
    + ROC 곡선이 굽어지면 굽어질수록 AUC가 넓어지므로, 더욱 정확한 모델임
         
참고자료 : https://taeguu.tistory.com/32 
/ http://contents.kocw.net/KOCW/document/2014/Chungbuk/choisanghyun/6.pdf

# 제출

In [ ]:
# Round predictions to nearest integer
preds=np.round(preds).astype(int)

In [ ]:
# Sample submission (to get right format)
sub=pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

# Add predictions
sub['Transported']=preds

# Replace 0 to False and 1 to True
sub=sub.replace({0:False,1:True})

# Prediction distribution
plt.figure(figsize=(6,6))
sub['Transported'].value_counts().plot.pie(explode=[0.1,0.1], autopct='%1.1f%%', shadow=True, textprops={'fontsize':16}).set_title("Prediction distribution")

In [ ]:
# Output to csv
sub.to_csv('submission.csv', index=False)

# 출처
SAMUEL CORTINHAS /Spaceship Titanic: A complete guide/ https://www.kaggle.com/code/samuelcortinhas/spaceship-titanic-a-complete-guide#Libraries